### 一、GPU 執行 Keras 與 TensorFlow 的基本架構

    Keras -> TensorFlow-gpu -> CUNA and cuDNN -> GPU

- **Keras**: TensorFlow 的高階 API，透過 TensorFlow 存取 GPU。



- **TensorFlow-gpu**: TensorFlow 的 GPU 版本。



- **CUDA(Compute Unified Device Architecture)**: NVIDIA 的平行運算架構。可以運用繪圖處理單元(GPU)的強大處理能力，增加運算效能。



- **cuDNN(CUDA Deep Neural Network library)**: 為 NVIDIA 深度學習軟體發展工具(SDK)的一部份，也是 GPU 的深度學習程式庫。

### 二、Windows 系統安裝 Keras、TensorFlow-GPU 的流程
#### 1. 確認顯示卡

- 查看已安裝的顯示卡資訊:

      開始 -> 系統資訊 -> 元件 -> 顯示

- 檢查顯示卡是否支援 CUDA

      顯示卡規格說明: 
      (https://www.geforce.com/hardware/technology/cuda/supported-gpus?field_gpu_type_value=All&page=4)


- 檢查支援 CUDA 的顯示卡它的計算能力是否滿足 tensorflow-gpu 的要求

      - 顯示卡的計算能力表: (https://developer.nvidia.com/cuda-gpus) 

      - tensorflow-gpu 硬體需求: NVIDIA® GPU card with CUDA® Compute Capability 3.5 or higher.
        (https://www.tensorflow.org/install/gpu)

[註] 顯示卡的驅動更新: (https://www.nvidia.com/Download/index.aspx?lang=en-us)

#### 2. 安裝 CUDA (官方安裝指南: https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/)

     (1) 找出對應的版本並下載 CUDA: (https://developer.nvidia.com/cuda-downloads)
     
     (2) 檢查 CUDA 的環境變數是否設定

#### 3. 安裝 CuDNN

     (1) 找出對應的版本並下載 CuDNN: (https://developer.nvidia.com/rdp/cudnn-download)

     (2) 下載後壓縮檔(zip 檔)裡的 cuda 目錄複製到已建立的 C:\tools 目錄底下
     
     (3) 在 C:\tools\cuda\bin 目錄裡有 cudnn64_7.dll 動態連結程式庫，將這個路徑加到環境變數中

#### 4. 建立和啟動虛擬環境

     建立虛擬環境: conda create --name [虛擬環境名稱] python=3.5 anaconda

     啟動虛擬環境: activate [虛擬環境名稱]

#### 5. 安裝 tensorflow-gpu

     pip install tensorflow-gpu
     
     [註] 測試 tensorflow-gpu:
           
          from tensorflow.python.client import device_lib
          print(device_lib.list_local_devices())
     
#### 6. 安裝 keras

     pip install keras

### 三、以 TensorFlow-GPU 執行矩陣運算

In [ ]:
import tensorflow as tf
from time import time

size = 100

# 使用 tf.device 指定 CPU 或 GPU
# with tf.device("/cpu:0"):
with tf.device("/gpu:0"):
    X = tf.random_normal([size, size], name = 'X')
    W = tf.random_normal([size, size], name = 'W')
    mul = tf.matmul(X, W, name = 'mul')
    sum_result = tf.reduce_sum(mul, name = 'sum')

# 設定 Session 的 config 參數來顯示 GPU 裝置的相關資訊
''' 
tfconfig = tf.ConfigProto(log_device_placement = True)
with tf.Session(config = tfconfig) as sess:
'''
with tf.Session() as sess:
    result = sess.run(sum_result)
    
print('result =', result)

### 四、測試 GPU 與 CPU 執行的效能

#### 1. 建立 performance_test 函數

In [ ]:
def performance_test(device_name, size):
    with tf.device(device_name):
        X = tf.random_normal([size, size], name = 'X')
        W = tf.random_normal([size, size], name = 'W')
        mul = tf.matmul(X, W, name = 'mul')
        sum_result = tf.reduce_sum(mul, name = 'sum')
        
    startTime = time()
    
    tfconfig = tf.ConfigProto(log_device_placement = True)
    with tf.Session(config = tfconfig) as sess:
        result = sess.run(sum_result)
        
    duration = time() - startTime
    
    print(device_name, " size = ", size, " Time: ", takeTimes)
    
    return takeTimes

#### 2. 執行 performance_test 函數

In [ ]:
# 初始化 i_set, cpu_set, gpu_set
i_set = []
cpu_set = []
gpu_set = []

for i range(0, 5000, 500):
    c = performance_test("/cpu:0", i)
    g = performance_test("/gpu:0", i)
    
    i_set.append(i)
    cpu_set.append(c)
    gpu_set.append(g)

#### 3. 以圖形檢視 CPU  和 GPU 的執行時間

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.gcf()
fig.set_size_inches(6, 4)
plt.plot(i_set, cpu_set, label = 'cpu')
plt.plot(i_set, gpu_set, label = 'gpu')
plt.legend()